# Introduction to Tensorflow

[Tensorflow](https://www.tensorflow.org]) is an open-source library for numerical computations on graphs. It can be used to specify and to train deep neural networks.

In [ ]:
import tensorflow as tf

# Specifying the Computational Graph

Let's set up the graph for the following neural network.

![](neural_network.png)

In [ ]:
# Define a placeholder for the inputs
x = tf.placeholder(tf.float32, shape=(None, 2))

# First hidden layer
w1 = tf.constant(
    [[-1., 1.], 
     [ 2., 0.]]
)
b1 = tf.constant([0., 0.])
h1 = tf.nn.relu(tf.matmul(x, w1) + b1)

# Output layer
w2 = tf.constant(
    [[1.],
     [-1.]]
)
b2 = tf.constant(0.)
y_pred = tf.matmul(h1, w2) + b2

y_pred

In Tensorflow, the graph is only evaluated when we call `tf.Session.run()`. We can look at the value of any node in the graph. We have to be sure to specify any `tf.placeholder` values in the graph, using the `feed_dict=` argument.

In [ ]:
with tf.Session() as sess:
    print(sess.run(y_pred, feed_dict={
        x: [[-1., 1.], 
            [ 1., 2.],
            [ 0., 1.]]
    }))

To make the neural network trainable, we need to turn the weights and biases into `tf.Variable` objects.

In [ ]:
# Define a placeholder for the inputs
x = tf.placeholder(tf.float32, shape=(None, 2))

# First hidden layer
w1 = tf.Variable(
    [[-1., 1.], 
     [ 2., 0.]]
)
b1 = tf.Variable(tf.zeros(2))
h1 = tf.nn.relu(tf.matmul(x, w1) + b1)

# Output layer
w2 = tf.Variable(
    [[1.],
     [-1.]]
)
b2 = tf.Variable(0.)
y_pred = tf.matmul(h1, w2) + b2

w1

In [ ]:
with tf.Session() as sess:
    # With tf.Variables, we need to explicitly initialize them
    # within a session.
    sess.run(tf.global_variables_initializer())
    print(sess.run(y_pred, feed_dict={
        x: [[-1., 1.], 
            [ 1., 2.],
            [ 0., 1.]]
    }))

# Training the Network

To train a network, we also need the true labels and a loss function that measures the discrepancy between the predicted and true labels.

In [ ]:
y_true = tf.placeholder(tf.float32, shape=None)
loss = tf.reduce_mean((y_pred - y_true) ** 2)

Now we need a way to optimize this loss function. Let's use gradient descent with a learning rate of $\eta = 0.25$.

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.25)
train = optimizer.minimize(loss)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # To force the model to train, we pass `train` to sess.run().
    # We also pass `loss` to sess.run() so that we can see the loss.
    print(sess.run((train, loss), feed_dict={
        x: [[-1., 1.]],
        y_true: 1
    }))
    
    # The weights should now be updated.
    print(sess.run(w2))
    
    # TODO: Predict the output for x = (1, 1) using the current weights.
    

# Feeding in Data

So far, we have used `tf.placeholder` and `feed_dict=` to pass data into the model. This still works with real data.

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("~/data400_share/2016-election.csv")
x_train = data[["at_least_bachelor_s_degree", "black"]].values
y_train = data["rep16_frac"].values

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # To force the model to train, we pass `train` to sess.run().
    # We also pass `loss` to sess.run() so that we can see the loss.
    print(sess.run((train, loss), feed_dict={
        x: x_train.astype(np.float32),
        y_true: y_train.astype(np.float32)
    }))

**Exercise:** Can you rewrite the above code to make Tensorflow process one observation at a time, instead of all of the observations at once? What about a batch of 5 observations?

In [ ]:
# YOUR CODE HERE

There are [more sophisticated ways to feed data into a Tensorflow graph](https://www.tensorflow.org/guide/datasets).

- You can create an iterator object out of a Numpy array in memory using `tf.data.Dataset.from_tensor_slices()`.
- If your data set is really large, it is recommend that you convert your data to `TFRecord` format and load the data into a `TFRecordDataset` object.